In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Lendo DataSet e apresentando**

In [ ]:
df=pd.read_csv("/kaggle/input/corona-virus-report/covid_19_clean_complete.csv")
df

## **Apresentando Informações de Linhas e Colunas**

In [ ]:
print ('Quantidade de Linhas e Colunas: ', df.shape) # Demonstrando numero de linhas/colunas do dataframe
print ('Colunas: ', list(df.columns)) # Mostrando as colunas


df['Date'] = pd.to_datetime(df['Date'])
print ('\nTipo da Data:', df.Date.dtypes) # Trocando o tipo de informação que está na coluna de Date

## **Valores Estátisticos**

In [ ]:
print ( df.describe() ) # Valores estátisticos

d1=df.groupby("Country/Region")["Deaths"].max().reset_index() 
d1=d1.sort_values("Deaths",ascending=False).head(10) # 10 Paises com mais mortes
print ('\n',d1)

## **Provincias da China**

In [ ]:
provinces_china = df.loc[df['Country/Region'] == 'China']['Province/State'].unique() # Filtrando os dados que vem da China
china_data = df[df['Province/State'].isin(provinces_china)] # Criando um dataframe apenas com esses dados
china_filt = china_data[['Province/State', 'Confirmed', 'Active', 'Deaths', 'Recovered']] #Separando as informações que me interessam

print("\n", china_filt) 
top_regions = china_filt.groupby(['Province/State'])['Confirmed'].max().reset_index() # Ordenando regiões com maior numero de confirmações
top_regions = top_regions.sort_values('Confirmed', ascending = False).head(5) # Escolhendo as 5 maiores
print ('\n', top_regions) 


## **Colunas com dados Perdidos**

In [ ]:
# Função que tenha uma linha do banco de dados como argumento
def concat_province_country(row):
    if pd.notna(row['Province/State']):
        return f"{row['Country/Region']}_{row['Province/State']}"
    else:
        return row['Country/Region']

# Faça uma cópia do banco de dados para poder retirar informações sem perder o banco de dados original
df_concat = df.copy() # Banco de dados para concatenar e retirar apenas Country/Province
df_new = df.copy() # Banco de dados para retirar outras informações

df_concat = df.loc[df['Country/Region'] == 'China']
china_new_data = df[df['Province/State'].isin(df_concat)]


# Aplique, por meio do método “apply”, a função criada no item
df_concat = df_concat.apply(concat_province_country, axis=1)

# Apagando a coluna de province/state
df_new.drop(columns=['Province/State'], inplace=True)

In [ ]:
df_new['Country/Province'] = df_concat # Criando no banco de dados novos uma coluna com os valores concatenados
print(df_new)
df_new = df_new.loc[df_new['Country/Region'] == 'China'] # Filtrando valores do banco de dados para termos dados apenas da China

In [ ]:
china_new = df_new.dropna(subset=['Country/Province']) # Apagando da nova coluna os Valores NaN tendo assim uma tabela filtrada apenas com dados da china
china_new.drop(columns=['Country/Region'], inplace=True) # Apagando a coluna que não será mais utilizada
print(china_new)
    

## **Novo dataframe, arquivo Worldmeter**

In [ ]:
world_df = pd.read_csv("/kaggle/input/corona-virus-report/worldometer_data.csv") # Importando dados do novo arquivo
world_df

## **Filtrando as Informações Interessantes**

In [ ]:
world_filt = world_df[['Country/Region', 'Continent', 'Population']] # Filtrando Infos

world_filt

## **Criando novo dataframe com as informações desejadas**

In [ ]:
country_df = df[['Country/Region', 'Deaths']] # Separando do arquivo original um arquivo com os dados interessantes
country_df = country_df.groupby('Country/Region').sum().reset_index() # Agrupando por paises

country_data = pd.merge(country_df, world_filt, on='Country/Region', how='inner') # Mesclando os 2 dataframes

print(country_data)

## **Criando nova coluna com Mortes/População (Em Milhões)**

In [ ]:
country_data['Deaths/Population (In Millions)'] = (country_data['Deaths'] / country_data['Population']) * 10e6 # Nova tabela = formula

print (country_data)

## **Ranking, Top 10 Paises com mais Mortes/População **

In [ ]:
country_data.sort_values('Deaths/Population (In Millions)', ascending = False).head(10) # Mostrar os 10 paises com mais mortes/população